# Financial Wavelet Prediction - Backtesting & Evaluation Demo

This notebook demonstrates the comprehensive evaluation framework including:
- Backtesting with realistic transaction costs
- Walk-forward analysis
- Risk metrics calculation
- Market regime analysis
- Performance reporting and visualization

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import evaluation components
import sys
sys.path.append('..')

from src.evaluation.backtest_engine import BacktestEngine, BacktestConfig, WalkForwardBacktest
from src.evaluation.trading_simulator import TradingSimulator, TradingCosts, Order, OrderType, OrderSide
from src.evaluation.risk_analyzer import RiskAnalyzer
from src.evaluation.performance_reporter import PerformanceReporter
from src.evaluation.market_regime_analyzer import MarketRegimeAnalyzer

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

## 1. Load Market Data

In [ ]:
# Load SPY data
symbol = 'SPY'
start_date = '2018-01-01'
end_date = '2024-01-01'

print(f"Loading {symbol} data from {start_date} to {end_date}...")
data = yf.download(symbol, start=start_date, end=end_date)

# Prepare data
data.columns = [col.lower() for col in data.columns]
data['returns'] = data['close'].pct_change()

print(f"\nData shape: {data.shape}")
print(f"Date range: {data.index[0]} to {data.index[-1]}")

# Plot price and volume
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(data.index, data['close'], linewidth=2)
ax1.set_ylabel('Price ($)')
ax1.set_title(f'{symbol} Price History')
ax1.grid(True, alpha=0.3)

ax2.bar(data.index, data['volume'], alpha=0.5)
ax2.set_ylabel('Volume')
ax2.set_xlabel('Date')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Generate Mock Predictions

For demonstration, we'll create mock predictions. In practice, these would come from your trained models.

In [ ]:
# Split data
train_end = '2022-01-01'
test_data = data[data.index >= train_end].copy()

# Generate mock predictions (in practice, use your trained models)
np.random.seed(42)

# Create predictions with some signal
# Combine momentum, mean reversion, and noise
momentum = test_data['returns'].rolling(20).mean().shift(1)
mean_reversion = -(test_data['close'] / test_data['close'].rolling(50).mean() - 1) * 0.1
noise = np.random.normal(0, 0.001, len(test_data))

predictions = momentum.fillna(0) * 0.3 + mean_reversion.fillna(0) * 0.3 + noise
predictions = predictions.values

print(f"Test data shape: {test_data.shape}")
print(f"Predictions shape: {predictions.shape}")
print(f"\nPrediction statistics:")
print(f"  Mean: {np.mean(predictions):.6f}")
print(f"  Std:  {np.std(predictions):.6f}")
print(f"  Min:  {np.min(predictions):.6f}")
print(f"  Max:  {np.max(predictions):.6f}")

## 3. Define Trading Signal Generator

In [ ]:
def generate_trading_signals(predictions, data, threshold=0.001):
    """
    Generate trading signals from predictions
    """
    signals = np.zeros(len(predictions))
    
    # Basic signal generation
    signals[predictions > threshold] = 1  # Long
    signals[predictions < -threshold] = -1  # Short
    
    # Add trend filter
    sma_20 = data['close'].rolling(20).mean()
    sma_50 = data['close'].rolling(50).mean()
    trend = (sma_20 > sma_50).astype(int) * 2 - 1
    
    # Only trade in direction of trend
    for i in range(len(signals)):
        if signals[i] * trend.iloc[i] < 0:
            signals[i] = 0
            
    return signals

# Generate signals
signals = generate_trading_signals(predictions, test_data)

# Plot signals
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(test_data.index, test_data['close'], label='Price', linewidth=2)
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(test_data.index, signals, label='Signal', linewidth=1)
ax2.fill_between(test_data.index, 0, signals, alpha=0.3)
ax2.set_ylabel('Signal')
ax2.set_xlabel('Date')
ax2.set_ylim(-1.5, 1.5)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('Trading Signals')
plt.tight_layout()
plt.show()

# Signal statistics
print(f"Signal distribution:")
print(f"  Long signals:  {(signals > 0).sum()} ({(signals > 0).sum() / len(signals) * 100:.1f}%)")
print(f"  Short signals: {(signals < 0).sum()} ({(signals < 0).sum() / len(signals) * 100:.1f}%)")
print(f"  No position:   {(signals == 0).sum()} ({(signals == 0).sum() / len(signals) * 100:.1f}%)")

## 4. Run Backtest

In [ ]:
# Configure backtest
config = BacktestConfig(
    initial_capital=100000,
    position_size=0.1,  # 10% per position
    max_positions=5,
    commission=0.001,  # 0.1%
    slippage=0.0005,   # 0.05%
    stop_loss=0.02,    # 2% stop loss
    take_profit=0.05,  # 5% take profit
)

# Initialize and run backtest
engine = BacktestEngine(config)
results = engine.run_backtest(
    data=test_data,
    predictions=predictions,
    signal_generator=generate_trading_signals
)

# Display results
print("Backtest Results:")
print(f"  Final Equity: ${results['final_equity']:,.2f}")
print(f"  Total Return: {results['total_return']:.2%}")
print(f"  Max Drawdown: {results['max_drawdown']:.2%}")
print(f"  Sharpe Ratio: {results['sharpe_ratio']:.2f}")
print(f"\nTrade Statistics:")
for key, value in results['trade_statistics'].items():
    print(f"  {key}: {value}")

## 5. Analyze Risk Metrics

In [ ]:
# Initialize risk analyzer
risk_analyzer = RiskAnalyzer()

# Calculate returns from equity curve
equity_curve = results['equity_curve']
returns = equity_curve['equity'].pct_change().dropna()

# Analyze risk metrics
risk_metrics = risk_analyzer.analyze(
    returns=returns,
    trades=results['trades']
)

# Display risk metrics
print("Risk Metrics:")
print(f"\nReturns:")
print(f"  Total Return: {risk_metrics.total_return:.2%}")
print(f"  Annual Return: {risk_metrics.annualized_return:.2%}")
print(f"  Volatility: {risk_metrics.volatility:.2%}")
print(f"  Downside Vol: {risk_metrics.downside_volatility:.2%}")

print(f"\nRisk-Adjusted Returns:")
print(f"  Sharpe Ratio: {risk_metrics.sharpe_ratio:.2f}")
print(f"  Sortino Ratio: {risk_metrics.sortino_ratio:.2f}")
print(f"  Calmar Ratio: {risk_metrics.calmar_ratio:.2f}")

print(f"\nDrawdown Metrics:")
print(f"  Max Drawdown: {risk_metrics.max_drawdown:.2%}")
print(f"  Max DD Duration: {risk_metrics.max_drawdown_duration} days")
print(f"  Recovery Factor: {risk_metrics.recovery_factor:.2f}")

print(f"\nRisk Metrics:")
print(f"  VaR (95%): {risk_metrics.value_at_risk:.2%}")
print(f"  CVaR (95%): {risk_metrics.conditional_value_at_risk:.2%}")
print(f"  Skewness: {risk_metrics.skewness:.2f}")
print(f"  Kurtosis: {risk_metrics.kurtosis:.2f}")

print(f"\nTrading Metrics:")
print(f"  Win Rate: {risk_metrics.win_rate:.2%}")
print(f"  Profit Factor: {risk_metrics.profit_factor:.2f}")
print(f"  Kelly Criterion: {risk_metrics.kelly_criterion:.2%}")

## 6. Stress Testing

In [ ]:
# Run stress tests
stress_results = risk_analyzer.stress_test(returns)

# Display stress test results
stress_df = pd.DataFrame(stress_results).T
stress_df = stress_df.round(4)

print("Stress Test Results:")
print(stress_df)

# Visualize stress test results
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Total return under stress
ax1 = axes[0, 0]
scenarios = list(stress_results.keys())
returns_stress = [stress_results[s]['total_return'] for s in scenarios]
ax1.bar(scenarios, returns_stress, color=['red' if r < 0 else 'green' for r in returns_stress])
ax1.set_title('Total Return Under Stress')
ax1.set_ylabel('Return')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Max drawdown under stress
ax2 = axes[0, 1]
dd_stress = [stress_results[s]['max_drawdown'] for s in scenarios]
ax2.bar(scenarios, dd_stress, color='red')
ax2.set_title('Max Drawdown Under Stress')
ax2.set_ylabel('Drawdown')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Sharpe ratio under stress
ax3 = axes[1, 0]
sharpe_stress = [stress_results[s]['sharpe_ratio'] for s in scenarios]
ax3.bar(scenarios, sharpe_stress, color=['red' if s < 0 else 'green' for s in sharpe_stress])
ax3.set_title('Sharpe Ratio Under Stress')
ax3.set_ylabel('Sharpe Ratio')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

# VaR under stress
ax4 = axes[1, 1]
var_stress = [stress_results[s]['var_95'] for s in scenarios]
ax4.bar(scenarios, var_stress, color='orange')
ax4.set_title('VaR (95%) Under Stress')
ax4.set_ylabel('VaR')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Market Regime Analysis

In [ ]:
# Initialize regime analyzer
regime_analyzer = MarketRegimeAnalyzer(n_regimes=4)

# Identify market regimes
regime_data = regime_analyzer.identify_regimes(data, lookback=20)

# Analyze performance by regime
regime_performance = regime_analyzer.analyze_performance_by_regime(results, regime_data)

# Display regime performance
print("Performance by Market Regime:")
for regime, metrics in regime_performance.items():
    print(f"\n{regime}:")
    print(f"  Days in regime: {metrics['days_in_regime']}")
    print(f"  % of time: {metrics['pct_of_time']:.1%}")
    print(f"  Total return: {metrics['total_return']:.2%}")
    print(f"  Volatility: {metrics['volatility']:.2%}")
    print(f"  Sharpe ratio: {metrics['sharpe_ratio']:.2f}")
    print(f"  Max drawdown: {metrics['max_drawdown']:.2%}")
    print(f"  Win rate: {metrics['win_rate']:.2%}")

# Plot regime analysis
regime_analyzer.plot_regime_analysis(data, regime_data)

## 8. Generate Performance Report

In [ ]:
# Initialize performance reporter
reporter = PerformanceReporter(output_dir="../reports")

# Generate comprehensive report
report = reporter.generate_report(
    backtest_results=results,
    strategy_name="Demo_Strategy",
    save_report=True
)

# Display report summary
print("Report Summary:")
for key, value in report.summary.items():
    if isinstance(value, float):
        if 'return' in key or 'rate' in key:
            print(f"  {key}: {value:.2%}")
        else:
            print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

# Generate tearsheet
reporter.generate_tearsheet(report, "Demo_Strategy")
print("\nTearsheet saved to reports/Demo_Strategy_tearsheet.png")

## 9. Trading Simulation with Realistic Costs

In [ ]:
# Configure realistic trading costs
trading_costs = TradingCosts(
    commission_rate=0.001,    # 0.1% per trade
    commission_minimum=1.0,   # $1 minimum
    spread_cost=0.0001,      # 1 basis point
    market_impact=0.0005,    # 5 basis points for large orders
    borrowing_cost=0.02      # 2% annual for shorts
)

# Initialize trading simulator
simulator = TradingSimulator(
    initial_capital=100000,
    costs=trading_costs,
    max_positions=10,
    allow_short=True,
    use_leverage=False
)

# Simulate trading for first 100 days
sim_data = test_data.iloc[:100]
sim_signals = signals[:100]

for i, (date, row) in enumerate(sim_data.iterrows()):
    # Record state
    simulator.record_state(date)
    
    # Process any pending orders
    simulator.process_orders(row)
    
    # Generate new orders based on signals
    if sim_signals[i] != 0 and i > 0:
        # Calculate position size
        portfolio_value = simulator.get_portfolio_value()
        position_size = portfolio_value * 0.1  # 10% per position
        quantity = int(position_size / row['close'])
        
        if quantity > 0:
            order = Order(
                symbol='SPY',
                side=OrderSide.BUY if sim_signals[i] > 0 else OrderSide.SELL,
                quantity=quantity,
                order_type=OrderType.MARKET
            )
            simulator.place_order(order)

# Generate simulation report
sim_report = simulator.generate_report()

print("Trading Simulation Results:")
print(f"\nSummary:")
for key, value in sim_report['summary'].items():
    print(f"  {key}: {value}")

print(f"\nCost Analysis:")
for key, value in sim_report['cost_analysis'].items():
    print(f"  {key}: {value}")

# Plot equity curve from simulation
if len(simulator.equity_history) > 0:
    equity_df = pd.DataFrame(simulator.equity_history)
    
    plt.figure(figsize=(12, 6))
    plt.plot(equity_df['timestamp'], equity_df['portfolio_value'], label='Portfolio Value', linewidth=2)
    plt.fill_between(equity_df['timestamp'], equity_df['cash'], equity_df['portfolio_value'], 
                     alpha=0.3, label='Positions Value')
    plt.xlabel('Date')
    plt.ylabel('Value ($)')
    plt.title('Trading Simulation - Portfolio Value')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 10. Walk-Forward Analysis Demo

In [ ]:
# Configure walk-forward backtest
wf_config = BacktestConfig(
    initial_capital=100000,
    position_size=0.1,
    max_positions=5,
    commission=0.001,
    slippage=0.0005
)

# Initialize walk-forward engine
wf_engine = BacktestEngine(wf_config)
walk_forward = WalkForwardBacktest(
    backtest_engine=wf_engine,
    train_period=252,  # 1 year
    test_period=63,    # 3 months
    step_size=21       # 1 month
)

# Simple model trainer for demo
def simple_model_trainer(train_data):
    # Mock model that uses simple momentum
    class MockModel:
        def predict(self, test_data):
            # Simple momentum strategy
            returns = test_data['returns'].rolling(20).mean()
            return returns.fillna(0).values
    return MockModel()

# Run walk-forward analysis on subset of data
wf_data = data.iloc[-1000:]  # Last 1000 days for faster demo

print("Running walk-forward analysis...")
wf_results = walk_forward.run(
    data=wf_data,
    model_trainer=simple_model_trainer,
    signal_generator=generate_trading_signals
)

# Display walk-forward results
print(f"\nWalk-Forward Analysis Results:")
print(f"  Number of windows: {wf_results['num_windows']}")
print(f"\nOverall Statistics:")
for key, value in wf_results['overall_statistics'].items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

# Plot window returns
window_returns = [w['total_return'] for w in wf_results['window_results']]

plt.figure(figsize=(12, 6))
plt.bar(range(len(window_returns)), window_returns, 
        color=['green' if r > 0 else 'red' for r in window_returns])
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.xlabel('Window')
plt.ylabel('Return')
plt.title('Walk-Forward Analysis - Returns by Window')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Calculate consistency
profitable_windows = sum(1 for r in window_returns if r > 0)
print(f"\nProfitable windows: {profitable_windows}/{len(window_returns)} ({profitable_windows/len(window_returns)*100:.1f}%)")

## Summary

This notebook demonstrated the comprehensive evaluation framework including:

1. **Backtesting Engine**: Run backtests with realistic transaction costs and position management
2. **Risk Analysis**: Calculate comprehensive risk metrics including Sharpe, Sortino, Calmar ratios, VaR, CVaR, and more
3. **Stress Testing**: Evaluate strategy performance under various market stress scenarios
4. **Market Regime Analysis**: Identify market regimes and analyze performance in different conditions
5. **Trading Simulation**: Realistic simulation with detailed cost modeling
6. **Walk-Forward Analysis**: Out-of-sample testing with rolling windows
7. **Performance Reporting**: Comprehensive reports with visualizations and tearsheets

The evaluation framework provides:
- Realistic backtesting with transaction costs
- Comprehensive risk metrics
- Market regime-aware analysis
- Professional-grade performance reports
- Walk-forward validation for robustness

This completes Sprint 6 of the Financial Wavelet Prediction project!